# First Approach - save data index. Not Good enough, see the New Attempt Part

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# importing train data to learn
train = pd.read_csv("train.csv", 
                    parse_dates = True, low_memory = False, index_col = 'Date')


# importing test data to learn
test = pd.read_csv("test.csv", 
                    parse_dates = True, low_memory = False, index_col = 'Date')

# additional store data
store = pd.read_csv("store.csv", 
                    low_memory = False)
# time series as indexes
train.index

DatetimeIndex(['2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31', '2015-07-31', '2015-07-31',
               '2015-07-31', '2015-07-31',
               ...
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2013-01-01'],
              dtype='datetime64[ns]', name='Date', length=1017209, freq=None)

In [3]:
train.head(5)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [4]:
test.head(5)

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,
2015-09-17,1,1,4,1.0,1,0,0
2015-09-17,2,3,4,1.0,1,0,0
2015-09-17,3,7,4,1.0,1,0,0
2015-09-17,4,8,4,1.0,1,0,0
2015-09-17,5,9,4,1.0,1,0,0


In [5]:
train = train[::-1].copy()
test = test[::-1].copy()

In [6]:
train.head(5)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2013-01-01,1115,2,0,0,0,0,a,1
2013-01-01,1114,2,0,0,0,0,a,1
2013-01-01,1113,2,0,0,0,0,a,1
2013-01-01,1112,2,0,0,0,0,a,1
2013-01-01,1111,2,0,0,0,0,a,1


In [7]:
test.head(5)

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,
2015-08-01,41088,1115,6,1.0,0,0,1
2015-08-01,41087,1114,6,1.0,0,0,0
2015-08-01,41086,1113,6,1.0,0,0,0
2015-08-01,41085,1112,6,1.0,0,0,0
2015-08-01,41084,1111,6,1.0,0,0,0


In [8]:
from sklearn.model_selection import TimeSeriesSplit

In [9]:
time_series_cv = TimeSeriesSplit(5)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 8 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(1)
memory usage: 69.8+ MB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41088 entries, 2015-08-01 to 2015-09-17
Data columns (total 7 columns):
Id               41088 non-null int64
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null int64
dtypes: float64(1), int64(5), object(1)
memory usage: 2.5+ MB


In [12]:
# data extraction
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.weekofyear

# data extraction
test['Year'] = test.index.year
test['Month'] = test.index.month
test['Day'] = test.index.day
test['WeekOfYear'] = test.index.weekofyear


# adding new variable
#train['SalePerCustomer'] = train['Sales']/train['Customers']
#train['SalePerCustomer'].describe()

In [13]:
state_holidays = {'0': 0, 'a': 1, 'b': 2, 'c': 3}

train['StateHolidayInt'] = train['StateHoliday'].map(state_holidays).astype(int)
test['StateHolidayInt'] = test['StateHoliday'].map(state_holidays).astype(int)

In [14]:
X_train = train.copy()
X_train.drop(['Sales', 'StateHoliday'], axis = 1, inplace = True)

y_train  = train['Sales'].copy()

X_test = test.copy()
X_test.drop(['StateHoliday'], axis = 1, inplace = True)
print("Done")

Done


In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 11 columns):
Store              1017209 non-null int64
DayOfWeek          1017209 non-null int64
Customers          1017209 non-null int64
Open               1017209 non-null int64
Promo              1017209 non-null int64
SchoolHoliday      1017209 non-null int64
Year               1017209 non-null int64
Month              1017209 non-null int64
Day                1017209 non-null int64
WeekOfYear         1017209 non-null int64
StateHolidayInt    1017209 non-null int64
dtypes: int64(11)
memory usage: 93.1 MB


In [16]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41088 entries, 2015-08-01 to 2015-09-17
Data columns (total 11 columns):
Id                 41088 non-null int64
Store              41088 non-null int64
DayOfWeek          41088 non-null int64
Open               41077 non-null float64
Promo              41088 non-null int64
SchoolHoliday      41088 non-null int64
Year               41088 non-null int64
Month              41088 non-null int64
Day                41088 non-null int64
WeekOfYear         41088 non-null int64
StateHolidayInt    41088 non-null int64
dtypes: float64(1), int64(10)
memory usage: 3.8 MB


# Training

In [17]:
from tpot import TPOTRegressor

In [34]:
model = TPOTRegressor(generations=5, population_size=10, 
                      offspring_size=5, cv = time_series_cv, 
                      n_jobs = 1, scoring = 'neg_mean_squared_error',
                      verbosity = 4)

29 operators have been imported by TPOT.


In [35]:
start_time = time.time()
model.fit(X_train, y_train)
print('Training time: ', time.time() - start_time)

Optimization Progress:  17%|█▋        | 6/35 [09:34<1:01:50, 127.93s/pipeline]

Skipped pipeline #6 due to time out. Continuing to the next pipeline.


Optimization Progress:  34%|███▍      | 12/35 [15:08<44:17, 115.52s/pipeline] 

Skipped pipeline #12 due to time out. Continuing to the next pipeline.


Optimization Progress:  46%|████▌     | 16/35 [21:42<47:31, 150.10s/pipeline]

...

Optimization Progress:  80%|████████  | 28/35 [24:55<01:26, 12.42s/pipeline] 

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.


Optimization Progress: 100%|██████████| 35/35 [30:18<00:00, 94.35s/pipeline]

Skipped pipeline #33 due to time out. Continuing to the next pipeline.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.


Training time:  1852.527333021164


In [42]:
predictions = model.predict(X_test)

Imputing missing values in feature set


In [56]:
X_test_new = X_test.copy()
X_test_new.drop(["Id"], axis = 1, inplace = True)
X_test_new.head(5)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,Day,WeekOfYear,StateHolidayInt
Date,,,,,,,,,,
2015-08-01,1115,6,1.0,0,1,2015,8,1,31,0
2015-08-01,1114,6,1.0,0,0,2015,8,1,31,0
2015-08-01,1113,6,1.0,0,0,2015,8,1,31,0
2015-08-01,1112,6,1.0,0,0,2015,8,1,31,0
2015-08-01,1111,6,1.0,0,0,2015,8,1,31,0


In [57]:
X_train.head(5)

,Store,DayOfWeek,Customers,Open,Promo,SchoolHoliday,Year,Month,Day,WeekOfYear,StateHolidayInt
Date,,,,,,,,,,,
2013-01-01,1115,2,0,0,0,1,2013,1,1,1,1
2013-01-01,1114,2,0,0,0,1,2013,1,1,1,1
2013-01-01,1113,2,0,0,0,1,2013,1,1,1,1
2013-01-01,1112,2,0,0,0,1,2013,1,1,1,1
2013-01-01,1111,2,0,0,0,1,2013,1,1,1,1


# The New Attempt

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_store = pd.read_csv("store.csv")

/Users/sergeymakarychev/anaconda/envs/deloitte/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
closed_store_data = data_test["Id"][data_test["Open"] == 0].values
data_train.StateHoliday = data_train.StateHoliday.replace(0,'0')
data_test.StateHoliday = data_test.StateHoliday.replace(0,'0')

In [4]:
data_train['Year'] = data_train['Date'].apply(lambda x: int(x[:4]))
data_train['Month'] = data_train['Date'].apply(lambda x: int(x[5:7]))
data_train["HolidayBin"] = data_train.StateHoliday.map({"0": 0, "a": 1, "b": 1, "c": 1})

In [5]:
del data_train['Date']
del data_train['StateHoliday']

In [6]:
data_test['Year'] = data_test['Date'].apply(lambda x: int(x[:4]))
data_test['Month'] = data_test['Date'].apply(lambda x: int(x[5:7]))
data_test["HolidayBin"] = data_test.StateHoliday.map({"0": 0, "a": 1, "b": 1, "c": 1})

In [7]:
del data_test['Date']
del data_test['StateHoliday']

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [9]:
X_train = data_train.copy()
X_train.drop(["Sales", "Customers"], axis = 1, inplace = True)

y_train = data_train['Sales'].copy()

X_test = data_test.copy()
X_test.drop(["Id"], axis = 1, inplace = True)
#X_test["Open"].astype('int64')

In [10]:
X_train.head(5)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,HolidayBin
0,1,5,1,1,1,2015,7,0
1,2,5,1,1,1,2015,7,0
2,3,5,1,1,1,2015,7,0
3,4,5,1,1,1,2015,7,0
4,5,5,1,1,1,2015,7,0


In [11]:
X_test.head(5)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,HolidayBin
0,1,4,1.0,1,0,2015,9,0
1,3,4,1.0,1,0,2015,9,0
2,7,4,1.0,1,0,2015,9,0
3,8,4,1.0,1,0,2015,9,0
4,9,4,1.0,1,0,2015,9,0


# Merge

In [12]:
store = pd.read_csv("store.csv")

In [13]:
store.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [14]:
new_train = pd.merge(X_train, 
                     store[["Store", "StoreType", "Assortment", "CompetitionDistance"]], 
                     on = 'Store', how = 'left')
new_train.head(5)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,HolidayBin,StoreType,Assortment,CompetitionDistance
0,1,5,1,1,1,2015,7,0,c,a,1270.0
1,2,5,1,1,1,2015,7,0,a,a,570.0
2,3,5,1,1,1,2015,7,0,a,a,14130.0
3,4,5,1,1,1,2015,7,0,c,c,620.0
4,5,5,1,1,1,2015,7,0,a,a,29910.0


In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Open             41077 non-null float64
Promo            41088 non-null int64
SchoolHoliday    41088 non-null int64
Year             41088 non-null int64
Month            41088 non-null int64
HolidayBin       41088 non-null int64
dtypes: float64(1), int64(7)
memory usage: 2.5 MB


In [16]:
new_test = pd.merge(X_test, 
                    store[["Store", "StoreType", "Assortment", "CompetitionDistance"]], 
                    on = 'Store', how = 'left')
new_test.head(5)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,HolidayBin,StoreType,Assortment,CompetitionDistance
0,1,4,1.0,1,0,2015,9,0,c,a,1270.0
1,3,4,1.0,1,0,2015,9,0,a,a,14130.0
2,7,4,1.0,1,0,2015,9,0,a,c,24000.0
3,8,4,1.0,1,0,2015,9,0,a,a,7520.0
4,9,4,1.0,1,0,2015,9,0,a,c,2030.0


In [17]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1112 non-null float64
CompetitionOpenSinceMonth    761 non-null float64
CompetitionOpenSinceYear     761 non-null float64
Promo2                       1115 non-null int64
Promo2SinceWeek              571 non-null float64
Promo2SinceYear              571 non-null float64
PromoInterval                571 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [18]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 11 columns):
Store                  1017209 non-null int64
DayOfWeek              1017209 non-null int64
Open                   1017209 non-null int64
Promo                  1017209 non-null int64
SchoolHoliday          1017209 non-null int64
Year                   1017209 non-null int64
Month                  1017209 non-null int64
HolidayBin             1017209 non-null int64
StoreType              1017209 non-null object
Assortment             1017209 non-null object
CompetitionDistance    1014567 non-null float64
dtypes: float64(1), int64(8), object(2)
memory usage: 93.1+ MB


In [19]:
new_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 11 columns):
Store                  41088 non-null int64
DayOfWeek              41088 non-null int64
Open                   41077 non-null float64
Promo                  41088 non-null int64
SchoolHoliday          41088 non-null int64
Year                   41088 non-null int64
Month                  41088 non-null int64
HolidayBin             41088 non-null int64
StoreType              41088 non-null object
Assortment             41088 non-null object
CompetitionDistance    40992 non-null float64
dtypes: float64(2), int64(7), object(2)
memory usage: 3.8+ MB


# TPOT Solution

In [12]:
from tpot import TPOTRegressor
from sklearn.model_selection import TimeSeriesSplit

In [13]:
time_series_cv = TimeSeriesSplit(5)

In [33]:
model = TPOTRegressor(generations=5, population_size=10, 
                      offspring_size=5, cv = time_series_cv, 
                      n_jobs = 1, scoring = 'neg_mean_squared_error',
                      verbosity = 4)

29 operators have been imported by TPOT.


In [ ]:
start_time = time.time()
model.fit(X_train, y_train)
print('Training time: ', time.time() - start_time)

Optimization Progress:   0%|          | 0/35 [00:00<?, ?pipeline/s]

In [ ]:
predictions = model.predict(X_test)

In [25]:
predictions

array([ 5892.70491497,  7845.05919573,  9425.29875259, ...,
        7289.24755421, 16261.44044784,  9300.66151509])

In [27]:
predictions[data_test["Open"] == 0] = 0

In [28]:
predictions[predictions < 0] = 0

In [29]:
predictions

array([ 5892.70491497,  7845.05919573,  9425.29875259, ...,
        7289.24755421, 16261.44044784,  9300.66151509])

In [32]:
output = pd.DataFrame({'Id': data_test['Id'], 'Sales': predictions})
output.to_csv('tpot_without_customers_new.csv', index = False)

# Catboost Solution

In [15]:
from catboost import CatBoostRegressor

In [16]:
catboost = CatBoostRegressor(iterations=1500, depth = 5)

In [17]:
catboost.fit(X_train, y_train)

0:	learn: 6759.7908661	total: 213ms	remaining: 5m 19s
1:	learn: 6586.0635645	total: 347ms	remaining: 4m 20s
2:	learn: 6418.3104945	total: 506ms	remaining: 4m 12s
3:	learn: 6256.3548900	total: 641ms	remaining: 3m 59s
4:	learn: 6100.0453017	total: 792ms	remaining: 3m 56s
5:	learn: 5949.2216880	total: 926ms	remaining: 3m 50s
6:	learn: 5803.7306766	total: 1.08s	remaining: 3m 50s
7:	learn: 5663.4188074	total: 1.21s	remaining: 3m 46s
8:	learn: 5528.1450770	total: 1.37s	remaining: 3m 46s
9:	learn: 5397.7674527	total: 1.5s	remaining: 3m 44s
10:	learn: 5272.1515093	total: 1.65s	remaining: 3m 43s
11:	learn: 5151.1534551	total: 1.79s	remaining: 3m 41s
12:	learn: 5034.6512983	total: 2.01s	remaining: 3m 49s
13:	learn: 4922.5023910	total: 2.22s	remaining: 3m 55s
14:	learn: 4814.5963535	total: 2.38s	remaining: 3m 55s
15:	learn: 4710.8123381	total: 2.53s	remaining: 3m 55s
16:	learn: 4611.0122788	total: 2.71s	remaining: 3m 56s
17:	learn: 4515.0969003	total: 2.88s	remaining: 3m 57s
18:	learn: 4422.94331

149:	learn: 2490.5307057	total: 22.2s	remaining: 3m 19s
150:	learn: 2490.2094206	total: 22.3s	remaining: 3m 19s
151:	learn: 2489.9911337	total: 22.5s	remaining: 3m 19s
152:	learn: 2489.7431304	total: 22.6s	remaining: 3m 19s
153:	learn: 2489.4367293	total: 22.9s	remaining: 3m 19s
154:	learn: 2488.8645664	total: 23s	remaining: 3m 19s
155:	learn: 2488.6435816	total: 23.1s	remaining: 3m 19s
156:	learn: 2487.9553532	total: 23.3s	remaining: 3m 19s
157:	learn: 2487.7688391	total: 23.4s	remaining: 3m 19s
158:	learn: 2487.0212703	total: 23.6s	remaining: 3m 18s
159:	learn: 2486.7410188	total: 23.7s	remaining: 3m 18s
160:	learn: 2486.1290096	total: 23.9s	remaining: 3m 18s
161:	learn: 2485.8694438	total: 24s	remaining: 3m 18s
162:	learn: 2485.5534217	total: 24.1s	remaining: 3m 17s
163:	learn: 2484.8554623	total: 24.3s	remaining: 3m 17s
164:	learn: 2484.5456627	total: 24.4s	remaining: 3m 17s
165:	learn: 2483.8686451	total: 24.6s	remaining: 3m 17s
166:	learn: 2483.6350572	total: 24.7s	remaining: 3m 

299:	learn: 2451.2692477	total: 43.9s	remaining: 2m 55s
300:	learn: 2451.1863089	total: 44s	remaining: 2m 55s
301:	learn: 2450.9988679	total: 44.1s	remaining: 2m 55s
302:	learn: 2450.8061305	total: 44.3s	remaining: 2m 55s
303:	learn: 2450.5229974	total: 44.5s	remaining: 2m 55s
304:	learn: 2450.4469019	total: 44.6s	remaining: 2m 54s
305:	learn: 2450.3619998	total: 44.8s	remaining: 2m 54s
306:	learn: 2450.1559268	total: 44.9s	remaining: 2m 54s
307:	learn: 2449.9218048	total: 45.1s	remaining: 2m 54s
308:	learn: 2449.8321364	total: 45.2s	remaining: 2m 54s
309:	learn: 2449.7434377	total: 45.4s	remaining: 2m 54s
310:	learn: 2449.6751994	total: 45.5s	remaining: 2m 54s
311:	learn: 2449.0349820	total: 45.7s	remaining: 2m 53s
312:	learn: 2448.7839518	total: 45.8s	remaining: 2m 53s
313:	learn: 2448.7116747	total: 45.9s	remaining: 2m 53s
314:	learn: 2448.6592725	total: 46.1s	remaining: 2m 53s
315:	learn: 2448.5579210	total: 46.2s	remaining: 2m 53s
316:	learn: 2448.4588378	total: 46.4s	remaining: 2

447:	learn: 2427.8119724	total: 1m 5s	remaining: 2m 32s
448:	learn: 2427.6744302	total: 1m 5s	remaining: 2m 32s
449:	learn: 2427.4818943	total: 1m 5s	remaining: 2m 32s
450:	learn: 2427.3524861	total: 1m 5s	remaining: 2m 32s
451:	learn: 2427.2619741	total: 1m 5s	remaining: 2m 32s
452:	learn: 2427.2019718	total: 1m 5s	remaining: 2m 32s
453:	learn: 2427.1740911	total: 1m 5s	remaining: 2m 32s
454:	learn: 2426.8991361	total: 1m 6s	remaining: 2m 31s
455:	learn: 2426.7875404	total: 1m 6s	remaining: 2m 31s
456:	learn: 2426.7448230	total: 1m 6s	remaining: 2m 31s
457:	learn: 2426.5259003	total: 1m 6s	remaining: 2m 31s
458:	learn: 2426.4963308	total: 1m 6s	remaining: 2m 31s
459:	learn: 2426.4114281	total: 1m 6s	remaining: 2m 31s
460:	learn: 2426.2414581	total: 1m 6s	remaining: 2m 30s
461:	learn: 2426.1817337	total: 1m 7s	remaining: 2m 30s
462:	learn: 2426.1100227	total: 1m 7s	remaining: 2m 30s
463:	learn: 2425.8791799	total: 1m 7s	remaining: 2m 30s
464:	learn: 2425.7222256	total: 1m 7s	remaining:

592:	learn: 2411.1377762	total: 1m 27s	remaining: 2m 13s
593:	learn: 2411.0445123	total: 1m 27s	remaining: 2m 13s
594:	learn: 2411.0022646	total: 1m 27s	remaining: 2m 13s
595:	learn: 2410.9253876	total: 1m 27s	remaining: 2m 13s
596:	learn: 2410.7178493	total: 1m 28s	remaining: 2m 13s
597:	learn: 2410.7003448	total: 1m 28s	remaining: 2m 13s
598:	learn: 2410.5049860	total: 1m 28s	remaining: 2m 13s
599:	learn: 2410.4621543	total: 1m 28s	remaining: 2m 12s
600:	learn: 2410.4146326	total: 1m 28s	remaining: 2m 12s
601:	learn: 2410.3420292	total: 1m 28s	remaining: 2m 12s
602:	learn: 2410.2872365	total: 1m 29s	remaining: 2m 12s
603:	learn: 2410.2365435	total: 1m 29s	remaining: 2m 12s
604:	learn: 2410.1340575	total: 1m 29s	remaining: 2m 12s
605:	learn: 2410.1132298	total: 1m 29s	remaining: 2m 12s
606:	learn: 2410.0679449	total: 1m 29s	remaining: 2m 11s
607:	learn: 2410.0090764	total: 1m 29s	remaining: 2m 11s
608:	learn: 2409.9914345	total: 1m 29s	remaining: 2m 11s
609:	learn: 2409.9514658	total:

...

885:	learn: 2389.0231071	total: 2m 11s	remaining: 1m 30s
886:	learn: 2388.9482500	total: 2m 11s	remaining: 1m 30s
887:	learn: 2388.9307148	total: 2m 11s	remaining: 1m 30s
888:	learn: 2388.8697530	total: 2m 11s	remaining: 1m 30s
889:	learn: 2388.8023284	total: 2m 11s	remaining: 1m 30s
890:	learn: 2388.7069217	total: 2m 11s	remaining: 1m 30s
891:	learn: 2388.6170857	total: 2m 11s	remaining: 1m 29s
892:	learn: 2388.5771022	total: 2m 12s	remaining: 1m 29s
893:	learn: 2388.5501379	total: 2m 12s	remaining: 1m 29s
894:	learn: 2388.5155528	total: 2m 12s	remaining: 1m 29s
895:	learn: 2388.4958787	total: 2m 12s	remaining: 1m 29s
896:	learn: 2388.4775533	total: 2m 12s	remaining: 1m 29s
897:	learn: 2388.4359768	total: 2m 12s	remaining: 1m 29s
898:	learn: 2388.4099239	total: 2m 12s	remaining: 1m 28s
899:	learn: 2388.3159555	total: 2m 13s	remaining: 1m 28s
900:	learn: 2388.2778082	total: 2m 13s	remaining: 1m 28s
901:	learn: 2388.2129295	total: 2m 13s	remaining: 1m 28s
902:	learn: 2388.1904001	total:

1029:	learn: 2381.9256929	total: 2m 37s	remaining: 1m 11s
1030:	learn: 2381.8889946	total: 2m 37s	remaining: 1m 11s
1031:	learn: 2381.8728267	total: 2m 38s	remaining: 1m 11s
1032:	learn: 2381.8308901	total: 2m 38s	remaining: 1m 11s
1033:	learn: 2381.8224415	total: 2m 38s	remaining: 1m 11s
1034:	learn: 2381.7989719	total: 2m 38s	remaining: 1m 11s
1035:	learn: 2381.7396366	total: 2m 39s	remaining: 1m 11s
1036:	learn: 2381.7234417	total: 2m 39s	remaining: 1m 11s
1037:	learn: 2381.6772408	total: 2m 39s	remaining: 1m 11s
1038:	learn: 2381.6212759	total: 2m 40s	remaining: 1m 11s
1039:	learn: 2381.6042481	total: 2m 40s	remaining: 1m 11s
1040:	learn: 2381.5872037	total: 2m 41s	remaining: 1m 11s
1041:	learn: 2381.5290822	total: 2m 41s	remaining: 1m 11s
1042:	learn: 2381.4956660	total: 2m 42s	remaining: 1m 11s
1043:	learn: 2381.4643882	total: 2m 42s	remaining: 1m 10s
1044:	learn: 2381.4087556	total: 2m 42s	remaining: 1m 10s
1045:	learn: 2381.3888238	total: 2m 43s	remaining: 1m 10s
1046:	learn: 2

1175:	learn: 2376.0817106	total: 3m 9s	remaining: 52.1s
1176:	learn: 2376.0483389	total: 3m 9s	remaining: 52s
1177:	learn: 2376.0321809	total: 3m 9s	remaining: 51.8s
1178:	learn: 2376.0183497	total: 3m 9s	remaining: 51.6s
1179:	learn: 2375.9657581	total: 3m 9s	remaining: 51.5s
1180:	learn: 2375.9303031	total: 3m 10s	remaining: 51.3s
1181:	learn: 2375.9069798	total: 3m 10s	remaining: 51.2s
1182:	learn: 2375.8482269	total: 3m 10s	remaining: 51s
1183:	learn: 2375.8153623	total: 3m 10s	remaining: 50.9s
1184:	learn: 2375.7970189	total: 3m 10s	remaining: 50.7s
1185:	learn: 2375.7552162	total: 3m 10s	remaining: 50.5s
1186:	learn: 2375.7278045	total: 3m 10s	remaining: 50.4s
1187:	learn: 2375.6741565	total: 3m 11s	remaining: 50.2s
1188:	learn: 2375.6520418	total: 3m 11s	remaining: 50s
1189:	learn: 2375.6398721	total: 3m 11s	remaining: 49.9s
1190:	learn: 2375.5636503	total: 3m 11s	remaining: 49.7s
1191:	learn: 2375.5110286	total: 3m 11s	remaining: 49.5s
1192:	learn: 2375.4690272	total: 3m 11s	re

1321:	learn: 2371.4245488	total: 3m 33s	remaining: 28.8s
1322:	learn: 2371.3799749	total: 3m 33s	remaining: 28.6s
1323:	learn: 2371.3606495	total: 3m 34s	remaining: 28.5s
1324:	learn: 2371.3411099	total: 3m 34s	remaining: 28.3s
1325:	learn: 2371.3277552	total: 3m 34s	remaining: 28.1s
1326:	learn: 2371.3022095	total: 3m 34s	remaining: 28s
1327:	learn: 2371.2688084	total: 3m 35s	remaining: 27.8s
1328:	learn: 2371.2419266	total: 3m 35s	remaining: 27.7s
1329:	learn: 2371.2288477	total: 3m 35s	remaining: 27.5s
1330:	learn: 2371.2004101	total: 3m 35s	remaining: 27.4s
1331:	learn: 2371.1625280	total: 3m 36s	remaining: 27.3s
1332:	learn: 2371.0992637	total: 3m 36s	remaining: 27.1s
1333:	learn: 2371.0564469	total: 3m 36s	remaining: 27s
1334:	learn: 2371.0127802	total: 3m 36s	remaining: 26.8s
1335:	learn: 2370.9547737	total: 3m 37s	remaining: 26.6s
1336:	learn: 2370.9090240	total: 3m 37s	remaining: 26.5s
1337:	learn: 2370.8882623	total: 3m 37s	remaining: 26.3s
1338:	learn: 2370.8800944	total: 3m

1468:	learn: 2367.0052794	total: 4m 6s	remaining: 5.2s
1469:	learn: 2366.9806886	total: 4m 6s	remaining: 5.03s
1470:	learn: 2366.9495486	total: 4m 6s	remaining: 4.86s
1471:	learn: 2366.9239350	total: 4m 6s	remaining: 4.7s
1472:	learn: 2366.9125130	total: 4m 7s	remaining: 4.53s
1473:	learn: 2366.8914164	total: 4m 7s	remaining: 4.36s
1474:	learn: 2366.8811667	total: 4m 7s	remaining: 4.19s
1475:	learn: 2366.8486725	total: 4m 7s	remaining: 4.03s
1476:	learn: 2366.8413948	total: 4m 7s	remaining: 3.86s
1477:	learn: 2366.8078050	total: 4m 7s	remaining: 3.69s
1478:	learn: 2366.8019571	total: 4m 7s	remaining: 3.52s
1479:	learn: 2366.7582438	total: 4m 8s	remaining: 3.35s
1480:	learn: 2366.7022846	total: 4m 8s	remaining: 3.19s
1481:	learn: 2366.6779226	total: 4m 8s	remaining: 3.02s
1482:	learn: 2366.6536187	total: 4m 8s	remaining: 2.85s
1483:	learn: 2366.6291618	total: 4m 8s	remaining: 2.68s
1484:	learn: 2366.5859488	total: 4m 8s	remaining: 2.51s
1485:	learn: 2366.5789644	total: 4m 9s	remaining: 

In [18]:
predictions = catboost.predict(X_test)

In [28]:
predictions[data_test["Open"] == 0] = 0

In [32]:
predictions[predictions < 0] = 0

In [33]:
np.unique(predictions)

array([    0.        ,    51.37041865,    64.68525734, ...,
       16851.78112205, 16984.19960681, 17013.28478438])

In [34]:
output = pd.DataFrame({'Id': data_test['Id'], 'Sales': predictions})
output.to_csv('catboost_without_customers.csv', index = False)

# Auto_ML Approach

In [20]:
from auto_ml import Predictor

In [20]:
# column_descriptions = {
# #    'Date': 'date',
# #    'Sales': 'output',
#     'Open': 'categorical',
#     'StateHoliday': 'categorical',
#     'SchoolHoliday': 'categorical',
#     'DayOfWeek': 'categorical',
    
#     #'Store': 'categorical',
#     'Open': 'categorical',
# #    'Id': 'ignore'
# }

In [21]:
train_data_for_auto_ml = pd.concat([new_train, y_train], axis = 1)

In [22]:
train_data_for_auto_ml.head(3)

,Store,DayOfWeek,Open,Promo,SchoolHoliday,Year,Month,HolidayBin,StoreType,Assortment,CompetitionDistance,Sales
0,1,5,1,1,1,2015,7,0,c,a,1270.0,5263
1,2,5,1,1,1,2015,7,0,a,a,570.0,6064
2,3,5,1,1,1,2015,7,0,a,a,14130.0,8314


In [23]:
column_descriptions = {
    'Sales': 'output',
    'StoreType': 'categorical',
    'Assortment': 'categorical'
}

ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)

ml_predictor.train(train_data_for_auto_ml)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingRegressor to predict Sales
Started at:
201

[211] random_holdout_set_from_training_data's score is: -2136.473
[214] random_holdout_set_from_training_data's score is: -2135.486
[217] random_holdout_set_from_training_data's score is: -2132.749
[220] random_holdout_set_from_training_data's score is: -2127.81
[223] random_holdout_set_from_training_data's score is: -2123.42
[226] random_holdout_set_from_training_data's score is: -2116.453
[229] random_holdout_set_from_training_data's score is: -2109.467
[232] random_holdout_set_from_training_data's score is: -2105.191
[235] random_holdout_set_from_training_data's score is: -2101.698
[238] random_holdout_set_from_training_data's score is: -2096.415
[241] random_holdout_set_from_training_data's score is: -2093.191
[244] random_holdout_set_from_training_data's score is: -2090.021
[247] random_holdout_set_from_training_data's score is: -2087.591
[250] random_holdout_set_from_training_data's score is: -2084.107
[255] random_holdout_set_from_training_data's score is: -2081.088
[260] random

[1240] random_holdout_set_from_training_data's score is: -1514.093
[1260] random_holdout_set_from_training_data's score is: -1507.03
[1280] random_holdout_set_from_training_data's score is: -1499.905
[1300] random_holdout_set_from_training_data's score is: -1494.841
[1320] random_holdout_set_from_training_data's score is: -1489.268
[1340] random_holdout_set_from_training_data's score is: -1482.725
Heard KeyboardInterrupt. Stopping training, and using the best checkpointed GradientBoosting model
The number of estimators that were the best for this training dataset: 1340
The best score on the holdout set: -1482.725300259876
Finished training the pipeline!
Total training time:
1:02:31


Here are the results from our GradientBoostingRegressor
predicting Sales
Calculating feature responses, for advanced analytics.


/Users/sergeymakarychev/anaconda/envs/deloitte/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
+----+---------------------+--------------+-----------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|    | Feature Name        |   Importance |     Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAD |   FRI_MAD |
|----+---------------------+--------------+-----------+-------------------+-------------------+-----------+-----------+-----------+-----------|
|  7 | HolidayBin          |       0.0015 |    0.0864 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 14 | Assortment=b        |       0.0030 |  nan      |          nan      |          nan      |  nan      |  nan      |  nan      |  nan      |
|  5 | Year                |       0.0071 |    0.3885 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 11 | StoreType=b         |       0.0081 |  nan      |          nan   

In [24]:
predictions = ml_predictor.predict(new_test)
#predictions[data_test["Open"] == 0] = 0
#predictions[predictions < 0] = 0

In [25]:
predictions

[4840.646402297227,
 7810.298448634223,
 10171.18664452188,
 6155.192260409212,
 7442.634902476641,
 6056.688432858111,
 8186.366071776033,
 8122.309840428154,
 5842.427018644081,
 6190.443163819683,
 7532.160231311248,
 7745.318378373719,
 7218.635509012115,
 7807.265238883743,
 6410.161363117054,
 5510.25796130995,
 6279.509597996613,
 9686.295232860997,
 10409.410185095841,
 9578.179988289114,
 7732.681267644135,
 5986.622599953571,
 6439.688222329124,
 5545.9751447293365,
 8728.590352882024,
 9329.456057541891,
 9549.245693507477,
 6335.096103930581,
 5870.588859531539,
 5730.416093215014,
 6639.79671324504,
 9510.904307214696,
 7288.939646469433,
 6130.884102849067,
 5599.460798394709,
 8115.968366544225,
 4970.630760938979,
 7662.649093481614,
 5321.912534146929,
 7040.043117193003,
 7699.336681792296,
 6092.679373282558,
 7571.157327383984,
 7135.790681789511,
 6538.189169240764,
 6511.943362560746,
 7084.811656113973,
 11192.06244914575,
 6904.669509252899,
 7658.12385046293,
 

In [26]:
output = pd.DataFrame({'Id': data_test['Id'], 'Sales': predictions})
output.to_csv('auto_ml_without_customers_with_store_the_best.csv', index = False)